In [1]:
library(RPostgreSQL)
library(Matching)
library(tidyverse)

Loading required package: DBI
Loading required package: MASS
## 
##  Matching (Version 4.9-2, Build Date: 2015-12-25)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.1     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ dplyr::select() masks MASS::select()


In [2]:
data_dir <- file.path("..", "data")
sql_dir <- file.path("..", "sql")

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [4]:
ventfreedays <- dbGetQuery(con, "select icustay_id, ventfreeday28 from subgroup;")
head(ventfreedays)

icustay_id,ventfreeday28
200325,21.95833
200339,21.10000
200820,28.00000
201528,12.10417
201598,28.00000
201741,0.00000


In [5]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))

In [6]:
names(full_data)

[1] "icustay_id"                     "hadm_id"                       
  [3] "subject_id"                     "first_careunit"                
  [5] "intime"                         "outtime"                       
  [7] "angus"                          "age"                           
  [9] "icu_order"                      "echo_time"                     
 [11] "echo"                           "gender"                        
 [13] "weight"                         "saps"                          
 [15] "sofa"                           "elix_score"                    
 [17] "vent"                           "vaso"                          
 [19] "icu_adm_weekday"                "icu_adm_hour"                  
 [21] "mort_28_day"                    "icu_los_day"                   
 [23] "icd_chf"                        "icd_afib"                      
 [25] "icd_renal"                      "icd_liver"                     
 [27] "icd_copd"                       "icd_cad"                       
 [29] "icd_stroke"                     "icd_malignancy"                
 [31] "vs_temp_flag"                   "vs_temp_first"                 
 [33] "vs_temp_min"                    "vs_temp_max"                   
 [35] "vs_map_flag"                    "vs_map_first"                  
 [37] "vs_map_min"                     "vs_map_max"                    
 [39] "vs_cvp_flag"                    "vs_cvp_first"                  
 [41] "vs_cvp_min"                     "vs_cvp_max"                    
 [43] "vs_heart_rate_flag"             "vs_heart_rate_first"           
 [45] "vs_heart_rate_min"              "vs_heart_rate_max"             
 [47] "lab_hemoglobin_flag"            "lab_hemoglobin_first"          
 [49] "lab_hemoglobin_min"             "lab_hemoglobin_max"            
 [51] "lab_hemoglobin_abnormal"        "lab_wbc_flag"                  
 [53] "lab_wbc_first"                  "lab_wbc_min"                   
 [55] "lab_wbc_max"                    "lab_wbc_abnormal"              
 [57] "lab_lactate_flag"               "lab_lactate_first"             
 [59] "lab_lactate_min"                "lab_lactate_max"               
 [61] "lab_lactate_abnormal"           "lab_po2_flag"                  
 [63] "lab_po2_first"                  "lab_po2_min"                   
 [65] "lab_po2_max"                    "lab_po2_abnormal"              
 [67] "lab_troponin_flag"              "lab_troponin_first"            
 [69] "lab_troponin_min"               "lab_troponin_max"              
 [71] "lab_troponin_abnormal"          "lab_potassium_flag"            
 [73] "lab_potassium_first"            "lab_potassium_min"             
 [75] "lab_potassium_max"              "lab_potassium_abnormal"        
 [77] "lab_creatinine_kinase_flag"     "lab_creatinine_kinase_first"   
 [79] "lab_creatinine_kinase_min"      "lab_creatinine_kinase_max"     
 [81] "lab_creatinine_kinase_abnormal" "lab_creatinine_flag"           
 [83] "lab_creatinine_first"           "lab_creatinine_min"            
 [85] "lab_creatinine_max"             "lab_creatinine_abnormal"       
 [87] "lab_pco2_flag"                  "lab_pco2_first"                
 [89] "lab_pco2_min"                   "lab_pco2_max"                  
 [91] "lab_pco2_abnormal"              "lab_bnp_flag"                  
 [93] "lab_bnp_first"                  "lab_bnp_min"                   
 [95] "lab_bnp_max"                    "lab_bnp_abnormal"              
 [97] "lab_bicarbonate_flag"           "lab_bicarbonate_first"         
 [99] "lab_bicarbonate_min"            "lab_bicarbonate_max"           
[101] "lab_bicarbonate_abnormal"       "lab_bun_flag"                  
[103] "lab_bun_first"                  "lab_bun_min"                   
[105] "lab_bun_max"                    "lab_bun_abnormal"              
[107] "lab_platelet_flag"              "lab_platelet_first"            
[109] "lab_platelet_min"               "lab_platelet_max"              
[111] "lab_platelet_abnormal"          "la

In [7]:
set.seed(4958)

In [8]:
ps_matches <- Match(Y = NULL, Tr = full_data$echo_int, X = full_data$ps, M = 1,
                    estimand = "ATT", caliper = 0.01,
                    exact = FALSE, replace = FALSE)

In [9]:
matches_df <-
full_data[ps_matches$index.treated, "icustay_id", drop = FALSE] %>%
mutate(match = full_data[ps_matches$index.control, "icustay_id"])

head(matches_df)

icustay_id,match
289157,288615
284489,217156
251858,238836
269654,259409
281742,280230
238407,203972


In [10]:
result <- full_data %>%
left_join(ventfreedays, by = "icustay_id") %>%
left_join(matches_df, by = "icustay_id") %>%
select(icustay_id, echo_int, icu_los_day, mort_28_day_int, ventfreeday28, match, ps) %>%
setNames(c("icustay_id", "Echo", "ICU length of stay", "28 day mortality",
           "Ventilation free days (28 days)", "Match id", "Propensity score"))

head(result)

icustay_id,Echo,ICU length of stay,28 day mortality,Ventilation free days (28 days),Match id,Propensity score
201220,0,4.401343,1,0.00000,NA,0.4544669
215842,0,1.718438,0,28.00000,NA,0.1568838
234312,0,15.505602,0,18.58125,NA,0.3002571
289157,1,6.791667,0,28.00000,288615,0.6991412
211964,0,1.350949,0,28.00000,NA,0.3432285
230173,0,2.918218,1,0.00000,NA,0.2464983


In [11]:
summary(ps_matches)


Estimate...  0 
SE.........  0 
T-stat.....  NaN 
p.val......  NA 

Original number of observations..............  6162 
Original number of treated obs...............  3063 
Matched number of observations...............  1550 
Matched number of observations  (unweighted).  1550 

Number of obs dropped by 'exact' or 'caliper'  1513 



In [12]:
tab <- table(full_data$mort_28_day[ps_matches$index.treated],
             full_data$mort_28_day[ps_matches$index.control],
             dnn = c("Echo", "Control"))
tab

    Control
Echo   0   1
   0 817 349
   1 261 123

In [13]:
tab[2, 1] / tab[1, 2]
paste("95% Confint",
      round(exp(c(log(tab[2, 1] / tab[1, 2]) - qnorm(0.975) * sqrt(1 / tab[1, 2] + 1 / tab[2, 1]),
                  log(tab[2, 1] / tab[1, 2]) + qnorm(0.975) * sqrt(1 / tab[1, 2] + 1 / tab[2, 1]))), 7))

[1] 0.747851

[1] "95% Confint 0.6370274" "95% Confint 0.8779545"

In [14]:
mcnemar.test(tab)


	McNemar's Chi-squared test with continuity correction

data:  tab
McNemar's chi-squared = 12.408, df = 1, p-value = 0.0004275


In [15]:
data.table::fwrite(result, file.path(data_dir, "table-5-columns.csv"))